In [3]:
import pyspark
from pyspark.sql import SparkSession
from operator import add
import string
from pprint import pprint
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [4]:
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("Weilin_PartB")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/04 09:32:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/04 09:32:09 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


# B.1 Load the CSV file from HDFS

In [10]:
df = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.119:9000/parking-citations.csv')\
    .cache()

In [12]:
spark_context.uiWebUrl

'http://host-192-168-2-86-de1:4040'

In [14]:
df.count()

13077724

In [13]:
df.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|           200304|null|HOND|        PA|  

# B.2 Print the schema for the DataFrame

In [17]:
df.printSchema()
df.show()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)

+-------------+--------------------+----------

# B.3 Count the number of rows in the CSV file.

In [18]:
df.count()

13077724

# B.4 Count the number of partitions in the underlying RDD

In [19]:
df.rdd.getNumPartitions()

16

# B.5 Drop the columns VIN, Latitude and Longitude.

In [20]:
df=df.drop('VIN', 'Latitude', 'Longitude')
df.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|           200304|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|     

# B.6 Find the maximum fine amount and the number of fines have this amount

In [18]:
max_amount = df.withColumn("fine amount", col("fine amount").cast("float")).agg({"fine amount":"max"}).collect()[0][0]
num = df.where(df["fine amount"] == max_amount).count()

print("The maximum fine amount is {} and there are {} fines have this amount".format(max_amount, num))

The maximum fine amount is 1100.0 and there are 626 fines have this amount


# B.7 Show the top 20 most frequent vehicle makes, and their frequencies.

In [ ]:
df.groupBy("Make").count().orderBy("count", ascending=False).show(20)

# B.8 Create a new column and map the original colors to their corresponding values

In [7]:
COLORS = {
    'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black', 
    'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze', 
    'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold', 
    'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory', 
    'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon', 
    'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 
    'SL':'Silver', 'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 
    'WT':'White', 'WH':'White', 'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
}

In [6]:
# define a function for mapping the colors
def color_long(x):
    if x in COLORS:
        return COLORS[x]
    else:
        return x

In [8]:
# create a User Defined Function
ColorLongUDF = udf(color_long, StringType())

In [14]:
# create a new colunm
df = df.withColumn("Color Long", ColorLongUDF(col("Color")))

In [17]:
df.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+----------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|Color Long|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+----------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|        

# B.9 the most frequent colour value for Toyotas

In [25]:
freq_color = df.where(df.Make == "TOYT").groupBy("Make","Color Long").count().orderBy("count", ascending=False)
freq_color.show(5)
print("the most frequent color value for Toyotas is", freq_color.collect()[0][1])

+----+----------+------+
|Make|Color Long| count|
+----+----------+------+
|TOYT|      Gray|489697|
|TOYT|     White|434595|
|TOYT|     Black|353812|
|TOYT|    Silver|347894|
|TOYT|      Blue|180091|
+----+----------+------+
only showing top 5 rows



the most frequent color value for Toyotas is Gray
